<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/windows_data_prepr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [3]:
filename='/content/gdrive/My Drive/ML_proz/data_source_cleaned.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 57)

In [0]:
df['tenders.procuringEntity_address_postalCode']=df['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
regions=dict()
for key,value in table_postal_code.items():
    for item in value:
        regions[item]=key
def get_region(raw,regions):
    try:
        raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
    except:  
        raw['region']=np.nan
    return raw
df=df.apply(get_region,args=(regions,),axis=1)

In [0]:
class DataPreproccessing:
    def __init__(self, data_source, churn_interval=90, start_period=None, end_period=None,start_data_unique=None):
        if start_period and end_period:
          data_source=data_source[pd.to_datetime(data_source['new_date'])>=start_period]
          data_source=data_source[pd.to_datetime(data_source['new_date'])<end_period]
        self.start_period=start_period
        self.end_period=end_period
        self.start_data_unique=start_data_unique
        # сформували 2 вікна(X,y)
        self.last_day = pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = pd.to_datetime(self.end_period) - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['new_date']) < self.divide_data]
        
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['new_date']) >= self.divide_data]
        # вибираємо унікальних постачальників
#         self.unique_id_data=self.data_source[(pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique) & (pd.to_datetime(self.data_source['new_date'])<=self.divide_data)]
        self.unique_data=self.data_source[pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique]
        self.unique_data=self.unique_data[pd.to_datetime(self.unique_data['new_date'])<self.divide_data]
        self.unique_id = list(self.unique_data['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])
#         print(self.feature_data[self.feature_data['unique_id']=='2969913153'])    
    
    def print_dates(self):
        print('Features: {0}/{1}'.format(self.start_period,self.divide_data))
        print('Unique_id: {0}/{1}'.format(self.start_data_unique,self.divide_data))
        print('Labeling: {0}/{1}'.format(self.divide_data,self.end_period))
    
#     def activity_date(self):
#         f = ['winner', 'tenders.auctionPeriod_startDate', 'contracts.dateSigned', 'date_from_id', 'awards.date']
#         self.data_source['new_date'] = self.data_source[f].apply(
#             lambda x: x[f[2]] if x[f[0] == 1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]]), axis=1)
#         self.data_source['new_date'] = self.data_source[['new_date']].apply(lambda x: x['new_date'][:10], axis=1)
#         self.data_source['new_date'] = self.data_source[['new_date', 'date_from_id']].apply(
#             lambda x: x['date_from_id'] if x['new_date'] == '\\N' else x['new_date'], axis=1)

    def cleaning(self):
#         self.data_source['winner'] = self.data_source[['contracts.value_amount', 'contracts.status', 'winner']].apply \
#             (lambda x: 0 if x['contracts.value_amount'] == '\\N'
#                             or x['contracts.status'] == 'cancelled' else x['winner'], axis=1)
#         print('winner updated')
#         self.activity_date()
#         print('activity date')

#         def convert_to_float(raw):
#             try:
#                 raw['lots.value_amount'] = float(raw['lots.value_amount'])
#             except:
#                 raw['lots.value_amount'] = np.nan
#             return raw

#         self.data_source['lots.value_amount'] = self.data_source[['lots.value_amount']].apply(convert_to_float, axis=1)
#         print('convert lots')
        self.data_source['tenders.procuringEntity_address_postalCode']=self.data_source['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
        table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
        regions=dict()
        for key,value in table_postal_code.items():
            for item in value:
              regions[item]=key
        def get_region(raw,regions):
            
            try:
              raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
            except:  
              raw['region']=np.nan
            return raw
        self.data_source=self.data_source.apply(get_region,args=(regions,),axis=1)
        print('regions updated')
        
    def split_into_batches(self, n=20000, filename=None):
        
        self.cleaning()
        feature_batches = list()
        n_iter = int(np.ceil(self.feature_data.shape[0] / n))
        for i in range(n_iter):
            start_time = time.time()
            print('Batch {0}-{1}:'.format(i * n, (i + 1) * n))
            unique_id = self.feature_data.iloc[i * n:(i + 1) * n]['unique_id'].values
            feature_df = pd.DataFrame(unique_id, columns=['unique_id'])
            feature_batches.append(feature_df)
            # part of data_source
            df_ = self.data_source.loc[self.data_source['participants'].isin(unique_id)]
            print('Unique_supp:{0}-{1}'.format(len(unique_id), df_.shape[0]))
            df_ = df_.sort_values(by='new_date')
            self.divide_data = str(self.divide_data)[:10]

            # features:
            # 0. First activity
            # 1. Last activity
            def get_last_activity(raw, data,eps=0.7):
                date=data[data['participants'] == raw['unique_id']]
                dates=date['new_date'].values
                # favourite host
                try:
                  if date['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/date.shape[0] >=eps:
                    raw['favourite_host']=1
                  else:
                    raw['favourite_host']=0
                  raw['count_unique_hosts']=date['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
                except:
                  raw['favourite_host']=np.nan
                  raw['count_unique_hosts']=np.nan
                
                
#                 raw['unique_regions']=len(set(date['region'].values))
#                 raw['last_activity_date'] = dates[-1]
                try:
                    raw['first_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[0])).days
                    raw['last_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[-1])).days
                    if len(dates)!=1:
                      raw['average_activity']=(pd.to_datetime(dates[-1])-pd.to_datetime(dates[0])).days/(len(dates)-1)
                    else:
                      raw['average_activity']='?'
                except:
                    print('--------------')
                    raw['average_activity'] = '?'
                    
                return raw

            feature_batches[i] = feature_batches[i].apply(get_last_activity, args=(df_[['participants','new_date','tenders.procuringEntity_identifier_id','tenders.id']],0.75,),
                                                          axis=1)
            feature_batches[i]['last_activity_days'] = feature_batches[i].apply(
                lambda x: 0 if x['last_activity_days'] < 0 else x['last_activity_days'], axis=1)
            print(" --- %s seconds ---" % (time.time() - start_time))

            # 2. Average activity
#             def get_average_activity(raw, data):
#                 dates_ = list(data[data['participants'] == raw['unique_id']]['new_date'].values)
#                 try:
#                     raw['average_activity'] = (pd.to_datetime(dates_[-1]) - pd.to_datetime(dates_[0])).days / len(dates_)
#                 except:
#                     raw['average_activity'] = np.nan
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_average_activity, args=(df_[['participants','new_date']],),
#                                                           axis=1)

            # 3. Count lots
            freq = df_['participants'].value_counts()

            def count_lots(raw, frequency):
                raw['count_lots'] = frequency[raw['unique_id']]
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lots, args=(freq,), axis=1)

            # 4. Win, lose
            frequency = df_[df_['winner'] == 1]['participants'].value_counts()
            print(" --- %s seconds ---" % (time.time() - start_time))
            def win_lose(raw, frequency):
                # use try bsc if index not in frequency 'win'=0
                try:
                    raw['win'] = frequency[raw['unique_id']]
                except:
                    raw['win'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(win_lose, args=(
                frequency,), axis=1)
            feature_batches[i]['lose'] = feature_batches[i]['count_lots'] - feature_batches[i]['win']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             5. Count win open

            frequency = df_.loc[(df_['winner'] == 1) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_win_open(raw, frequency):
                try:
                    raw['win_open'] = frequency[raw['unique_id']]
                except:
                    raw['win_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_win_open, args=(
                frequency,), axis=1)
            feature_batches[i]['win_not_open'] = feature_batches[i]['win'] - feature_batches[i]['win_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             6. Count lose open
            frequency = df_.loc[(df_['winner'] == 0) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_lose_open(raw, frequency):
                try:
                    raw['lose_open'] = frequency[raw['unique_id']]
                except:
                    raw['lose_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lose_open, args=(
                frequency,), axis=1)
#             feature_batches[i]['lose_not_open'] = feature_batches[i]['lose'] - feature_batches[i]['lose_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
            # 7. Average economy
            data = df_[df_['winner'] == 1][
                ['participants', 'awards.value_amount', 'lots.value_amount']]
            data['economy_value'] = data['lots.value_amount'] - data['awards.value_amount']
            data['economy_percent'] = data['awards.value_amount'] / data['lots.value_amount']

            def get_economy(raw, data):
                d = data[data['participants'] == raw['unique_id']]
                raw['economy_value'] = np.nanmean(d['economy_value'])
                raw['average_winner_price']=np.nansum(d['awards.value_amount'])/d.shape[0]
                raw['economy_percent'] = np.nanmean(d['economy_percent'])
                return raw

            feature_batches[i] = feature_batches[i].apply(get_economy, args=(data,), axis=1)
#             8. Get label
            participants = list(self.label_data['participants'].value_counts().index)

            def get_y(raw, participants):
              if raw['unique_id'] in participants:
                  raw['y'] = 1
              else:
                  raw['y'] = 0
              return raw
#             9. Get favourite hosts
            feature_batches[i] = feature_batches[i].apply(get_y, args=(participants,), axis=1)
#             def favourite_percent(raw, data, eps = 0.7):
#                 d = data[data['participants']==raw['unique_id']].drop_duplicates('tenders.id')
#                 try:
#                   if d['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/d.shape[0] >=eps:
#                     raw['favourite_host']=1
#                   else:
#                     raw['favourite_host']=0
#                     raw['count_unique_hosts']=d['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
#                 except:
#                   raw['favourite_host']=np.nan
#                   raw['count_unique_hosts']=np.nan
#                 del d
#                 return raw
#             feature_batches[i] = feature_batches[i].apply(favourite_percent, args=(df_[['participants', 'tenders.id', 'tenders.procuringEntity_identifier_id']],0.75,), axis=1)
            # 10. Get unique region
            print(" Batch %s seconds ---" % (time.time() - start_time))
        if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename + '{0}'.format(i) + '.csv', index=False)

        
        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        self.feature_data = all_batches_df.copy()
        if filename:
            print('write to csv_all')
            all_batches_df.to_csv(filename + '_all.csv', index=False)
        print(" --- %s seconds ---" % (time.time() - start_time))
    def get_label(self):
        participants = list(self.label_data['participants'].value_counts().index)

        def get_y(raw, participants):
            if raw['unique_id'] in participants:
                raw['y'] = 1
            else:
                raw['y'] = 0
            return raw

        self.feature_data = self.feature_data.apply(get_y, args=(participants,), axis=1)


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','tenders.procurementMethod','contracts.status','awards.value_amount','tenders.id','tenders.procuringEntity_identifier_id','lots.value_amount']

In [7]:
prepr = DataPreproccessing(df.iloc[:100000],start_period='2016-09-10',end_period='2017-06-07',start_data_unique='2016-12-09',churn_interval=90)
prepr.print_dates()
prepr.split_into_batches(n=10000)
print(prepr.feature_data.head(10))
a = prepr.feature_data[prepr.feature_data['y'] == 0]['unique_id'].values
print(a)
print(len(a))

2017-03-09 00:00:00
Features: 2016-09-10/2017-03-09 00:00:00
Unique_id: 2016-12-09/2017-03-09 00:00:00
Labeling: 2017-03-09 00:00:00/2017-06-07
regions updated
Batch 0-10000:
Unique_supp:5742-11117
 --- 41.245128870010376 seconds ---
 --- 45.332091093063354 seconds ---
 --- 49.37126588821411 seconds ---
 --- 53.47612023353577 seconds ---
 --- 57.595839738845825 seconds ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:232: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:234: RuntimeWarning: Mean of empty slice


 Batch 84.98224949836731 seconds ---
Concat
 --- 84.98580431938171 seconds ---
  unique_id  favourite_host  count_unique_hosts  first_activity_days  \
0  40473930               0                  57                   52   
1  25394112               0                 112                  177   
2  23359034               0                  58                  171   
3  00131305               0                  47                  140   
4  21560766               0                  47                  169   
5  39273420               0                  51                  155   
6  32490244               0                  64                  171   
7  00131954               0                  37                  139   
8  39726851               0                  91                  174   
9  39537943               0                  30                  150   

   last_activity_days average_activity  count_lots  win  lose  win_open  \
0                   2         0.892857          57   

In [9]:
prepr1 = DataPreproccessing(df.iloc[:100000],start_period='2016-09-10',end_period='2017-09-05',start_data_unique='2017-03-09',churn_interval=90)
prepr1.print_dates()
prepr1.split_into_batches(n=10000)
print(prepr.feature_data.head(10))
prepr1.feature_data[prepr1.feature_data['unique_id'].isin(a)]

2017-06-07 00:00:00
Features: 2016-09-10/2017-06-07 00:00:00
Unique_id: 2017-03-09/2017-06-07 00:00:00
Labeling: 2017-06-07 00:00:00/2017-09-05
regions updated
Batch 0-10000:
Unique_supp:6567-15235
 --- 56.83044457435608 seconds ---
 --- 61.927780866622925 seconds ---
 --- 66.5454261302948 seconds ---
 --- 71.25546669960022 seconds ---
 --- 75.98118305206299 seconds ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:232: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:234: RuntimeWarning: Mean of empty slice


 Batch 110.23573756217957 seconds ---
Concat
 --- 110.23969101905823 seconds ---
  unique_id  favourite_host  count_unique_hosts  first_activity_days  \
0  40473930               0                  57                   52   
1  25394112               0                 112                  177   
2  23359034               0                  58                  171   
3  00131305               0                  47                  140   
4  21560766               0                  47                  169   
5  39273420               0                  51                  155   
6  32490244               0                  64                  171   
7  00131954               0                  37                  139   
8  39726851               0                  91                  174   
9  39537943               0                  30                  150   

   last_activity_days average_activity  count_lots  win  lose  win_open  \
0                   2         0.892857          57 

,unique_id,favourite_host,count_unique_hosts,first_activity_days,last_activity_days,average_activity,count_lots,win,lose,win_open,win_not_open,lose_open,economy_value,average_winner_price,economy_percent,y


In [11]:
prepr1.feature_data[prepr1.feature_data['unique_id'].isin(a)]

,unique_id,favourite_host,count_unique_hosts,first_activity_days,last_activity_days,average_activity,count_lots,win,lose,win_open,win_not_open,lose_open,economy_value,average_winner_price,economy_percent,y
309,2443215100,1,1,90,90,0,3,3,0,3,0,0,38783.666667,44999.666667,0.534710,0
744,35265531,0,3,163,90,36.5,3,2,1,1,1,1,4809.435000,643221.000000,0.995612,0
1156,3134114519,0,2,184,0,92,3,1,2,1,0,2,12001.000000,43999.000000,0.785696,1
1173,38513696,1,1,90,90,?,1,1,0,0,1,0,0.000000,199000.000000,1.000000,0
1186,2754520699,0,3,189,0,94.5,3,1,2,1,0,2,2908.000000,17792.000000,0.859517,1
1228,35840937,0,2,184,90,94,2,2,0,2,0,0,8572.020000,22830.980000,0.629316,1
1241,315609,1,1,90,90,?,1,1,0,0,1,0,0.000000,4140.000000,1.000000,0
1274,2816203543,0,2,184,90,94,2,1,1,1,0,1,21624.000000,41976.000000,0.660000,0
1348,1830610570,1,1,90,90,?,1,1,0,0,1,0,0.000000,24960.000000,1.000000,0
1359,393895226599,1,1,90,90,?,1,1,0,0,1,0,0.000000,125583.000000,1.000000,0


In [0]:
prepr..head(100)

,unique_id
0,25394112
1,32490244
2,31816235
3,37149148
4,39726851
5,2969913153
6,39190161
7,33614922
8,38385636
9,04947995
